In [25]:
# Import Library

import pandas as pd
import numpy as np
import pickle
import calendar

from tensorflow.keras.models import load_model
from keras import backend as K

In [29]:
# Custom Metrics

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [23]:
# Load the Models

with open('final_pipeline.pkl', 'rb') as file_1:
  model_pipeline = pickle.load(file_1)

# model_ann = load_model('churn_model.h5')
model_ann = load_model("churn_model.h5", custom_objects={"f1_m": f1_m})


In [24]:
# Create New Data

data_inf = {
    'user_id': '9f420209e7d129f3',
    'age': 29,
    'gender': 'M',
    'region_category': 'Town',
    'membership_category': 'Premium Membership',
    'joining_date': '2015-03-20',
    'joined_through_referral': 'Yes',
    'preferred_offer_types': 'Gift Vouchers/Coupons',
    'medium_of_operation': 'Smartphone',
    'internet_option': 'Wi-Fi',
    'last_visit_time': '22:29:49',
    'days_since_last_login': 10,
    'avg_time_spent': 56.67,
    'avg_transaction_value': 38604.69,
    'avg_frequency_login_days': 28.0,
    'points_in_wallet': 782.49,
    'used_special_discount': 'No',
    'offer_application_preference': 'Yes',
    'past_complaint': 'Yes',
    'complaint_status': 'Unsolved',
    'feedback': 'Poor Product Quality'
}

data_inf = pd.DataFrame([data_inf])
data_inf

,user_id,age,gender,region_category,membership_category,joining_date,joined_through_referral,preferred_offer_types,medium_of_operation,internet_option,...,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,9f420209e7d129f3,29,M,Town,Premium Membership,2015-03-20,Yes,Gift Vouchers/Coupons,Smartphone,Wi-Fi,...,10,56.67,38604.69,28.0,782.49,No,Yes,Yes,Unsolved,Poor Product Quality


In [26]:
# Preprocessing

data_inf["joining_date"] = pd.to_datetime(data_inf["joining_date"])
data_inf["last_visit_time"] = pd.to_datetime(data_inf["last_visit_time"])

data_inf['last_visit_time'] = pd.to_datetime(data_inf['last_visit_time'], format='%H:%M').dt.hour
data_inf['joining_year'] = pd.DatetimeIndex(data_inf['joining_date']).year
data_inf['joining_month'] = pd.DatetimeIndex(data_inf['joining_date']).month

data_inf['joining_month'] = data_inf['joining_month'].apply(lambda x: calendar.month_abbr[x])

data_inf['joining_year'] = data_inf['joining_year'].astype(str)

data_inf.drop(['user_id'], axis=1, inplace=True)
data_inf.drop(['joining_date'], axis=1, inplace=True)
data_inf.rename(columns = {'churn_risk_score':'churn'}, inplace = True)

In [27]:
# Transform Inference-Set

data_inf_transform = model_pipeline.transform(data_inf)
data_inf_transform

array([[-0.5099649 ,  1.5090489 , -0.27901747, -0.80009093,  0.52291084,
         1.43344791,  0.71661228,  0.        ,  1.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         1.        ,  0.        ,  0.        ,  1.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ]])

In [28]:
# Predict using Neural Network

y_pred_inf = model_ann.predict(data_inf_transform)
y_pred_inf = np.where(y_pred_inf >= 0.5, 1, 0)
y_pred_inf

1/1 [==============================] - 0s 247ms/step


array([[0]])

Dari data baru yang telah dibuat, customer tersebut terprediksi tidak churn.